In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
import os

In [0]:
files = os.listdir("drive/My Drive/ziknet-trends-rolling/data/Brazil/processed_data")
weeks_ahead = 1

In [4]:
datasets = {}
minNDictionary = {}
for file in files:
    datasets[file] = pd.read_csv("drive/My Drive/ziknet-trends-rolling/data/Brazil/processed_data/{}".format(file))
    datasets[file]["Searches"] /= 100
    print(file)
    print(datasets[file].tail())
    print("Number of observations: ", len(datasets[file]))

    

Bahia_2016-2017.csv
           Date  Searches  Cases
99   27/11/2017      0.26   19.0
100  04/12/2017      0.30   30.0
101  11/12/2017      0.09    1.0
102  18/12/2017      0.30    3.0
103  25/12/2017      0.28   56.0
Number of observations:  104
MatoGrosso_2016-2017.csv
           Date  Searches  Cases
99   27/11/2017      0.66    0.0
100  04/12/2017      0.00  -25.0
101  11/12/2017      0.55   19.0
102  18/12/2017      0.00    1.0
103  25/12/2017      0.68   -1.0
Number of observations:  104
MinasGerais_2016-2017.csv
           Date  Searches  Cases
99   27/11/2017      0.21    3.0
100  04/12/2017      0.17    3.0
101  11/12/2017      0.09    3.0
102  18/12/2017      0.16    9.0
103  25/12/2017      0.21    8.0
Number of observations:  104
RioDeJaneiro_2016-2017.csv
           Date  Searches  Cases
99   27/11/2017      0.22    0.0
100  04/12/2017      0.18    0.0
101  11/12/2017      0.20    0.0
102  18/12/2017      0.19    0.0
103  25/12/2017      0.23    0.0
Number of observations:

In [0]:
def series_to_supervised(df, outputColumn, n_in=1, n_out=1, dropnan=True):
    n_vars = df.shape[1]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("{}(t-{})".format(col, i)) for col in df.columns]

    # Append next observation[outputColumn] at n_out obs
    cols.append(df[outputColumn].shift(-n_out+1))
    names+=[outputColumn + "(t+{})".format(n_out-1)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [0]:
def getXY(dataset, state, weeksAhead):
    n_features = dataset.shape[1]
    
    n_weeks = 4
    reframed = series_to_supervised(dataset, "Cases",  n_weeks, weeksAhead)
    values = reframed.values
    
    totalFeatures = values.shape[1]

    x,y = values[:, :totalFeatures-1], values[:, totalFeatures-1] #Y is the last column, X is all the previous columns 

    x = x.reshape((x.shape[0], n_weeks, n_features)) # Reshape as 3-D
    return x, y

In [7]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Flatten, Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.layers.merge import concatenate
from keras.constraints import non_neg
def LSTM_NN_Model():
    input_layer = Input(shape=(4,2))
    
    
    b1_out = LSTM(16, return_sequences=False)(input_layer)

    b2_out = Dense(128, activation="relu", kernel_regularizer="l2")(input_layer)
    b2_out = Flatten()(b2_out)

    concatenated = concatenate([b1_out, b2_out])
    
    out = Dense(8, activation="relu", kernel_regularizer="l2")(concatenated)
    out = Dense(8, activation="relu", kernel_regularizer="l2")(out)
#     out = Dense(1, activation="linear", kernel_constraint=non_neg(), name='output_layer')(out)
    out = Dense(1, activation="linear", name='output_layer')(out)

    model = Model([input_layer], out)
    model.compile(loss=["mse"], optimizer=Adam(0.0001), metrics=["mae"])

    return model

Using TensorFlow backend.


In [8]:
for file in datasets:
    dataset = datasets[file]
    x, y = getXY(dataset[["Searches", "Cases"]], "", weeks_ahead)
    model = LSTM_NN_Model()
    
    xDim1 = x[0].shape[0]
    xDim2 = x[0].shape[1]
    
    splitIndex = len(x) - 52

    train_X = x[:splitIndex]
    train_y = y[:splitIndex]

    test_y = y[splitIndex:]
    outDataset = pd.DataFrame()
    outDataset["Observerd"] = test_y

    model.fit(
        train_X,
        train_y,
        epochs=200,
        batch_size=4,
        verbose=0,
        shuffle=False)

    predicted_y_history = []

    while(splitIndex < len(y)):
        predicted_y = \
            model.predict(x[splitIndex].reshape(1, xDim1, xDim2))[0]
        
        model.fit(
            x[:splitIndex+1],
            y[:splitIndex+1],
            epochs=40,
            batch_size=4,
            verbose=0,
            shuffle=False)

        predicted_y_history.append(predicted_y[0])
        splitIndex += 1
    
#     predicted_y_history = np.exp(predicted_y_history)
#     predicted_y_history -= minNDictionary[file]
    
#     test_y = np.exp(test_y)
#     test_y -= minNDictionary[file]
    
    outDataset["PREDICTED"] = predicted_y_history
    print("{} MSE: {}".format(file, mean_squared_error(test_y, predicted_y_history)))
    outDataset.to_csv(file)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Bahia_2016-2017.csv MSE: 545.2430897397248
MatoGrosso_2016-2017.csv MSE: 267.3387610989372
MinasGerais_2016-2017.csv MSE: 88.12514904008685
RioDeJaneiro_2016-2017.csv MSE: 1557.662827438306
SaoPaulo_2016-2017.csv MSE: 30.594698532882546
